In [ ]:
import tensorflow as tf
import keras
from keras import Sequential
from keras.layers import Dense
from embeddings.extracted_features_files import accounts_representations, extract_visual_features, extract_textual_features
from keras.initializers import RandomUniform
import calculate_competence_score
import numpy as np
import os
import pandas as pd
import cv2

In [ ]:
def calculate_accounts_data(textual_data_path, visual_data_path, account_list_path):
    account_data = pd.read_csv(account_list_path)
    account_list = account_data['influencers']
    account_concept = {}
    account_engagement = {}
    
    # Calculating engagement of accounts
    for idx, account in account_data.iterrows():
        account_engagement[account['influencer']] = calculate_competence_score.calculate_engagement_score(account['likes'], account['comments'], account['followers'])
    
    # Deriving concepts of the account
    for account in account_list:
        with open(textual_data_path + '/' + account, 'r') as text:
            text_embeddings = extract_textual_features.extract_textual_features(text)
        img = cv2.imread(visual_data_path + '/' + account,mode='RGB')
        image_embeddings = extract_visual_features.extract_and_flatten_features(img)
        account_concept[account] = accounts_representations.multimodal_social_account_embedding(text_embeddings, image_embeddings)
    
    
    return pd.DataFrame(account_concept, account_engagement)

In [ ]:
# Calculating engagement and concept of brands
brands_list_file = 'BRANDS_DATA_CSV'
brands_text_path = 'BRANDS_TEXT_DATA'
brands_image_path = 'BRANDS_IMAGE_DATA'

brands_data = calculate_accounts_data(brands_text_path, brands_image_path, brands_list_file)

In [ ]:
# Calculating engagement and concept of influencers
inf_list_file = 'INF_DATA_CSV'
inf_text_path = 'INF_TEXT_DATA'
inf_image_path = 'INF_IMAGE_DATA'

inf_data = calculate_accounts_data(inf_text_path, inf_image_path, inf_list_file)

In [ ]:
# Calculating similarity between brands and influencers
inf_brand_similarity = {}
for _, inf in inf_data.iterrows():
    for __, brand in brands_data.iterrows():
        inf_brand_similarity[inf['influencer']] = np.dot(inf[0:], brand[0:])

In [ ]:
# Influencer with most similar brands
k = 3

sorted_inf_brands = sorted(inf_brand_similarity.items(), key=lambda x: x[1], reverse=True)
k_highest_brands = [item[0] for item in inf_brand_similarity[:k]]
k_highest_brands

In [ ]:
# Calculating competence score
inf_competence_score = {}
for _, row in inf_brand_similarity:
    inf_competence_score[row['influencer']] = calculate_competence_score(
        row['engagement_score'],
        row['similarity_score']
    )

In [ ]:
def create_listnet_model(input_shape, num_nodes):
    """
    Create a ListNet neural network model using keras.

    Parameters:
        input_shape: Tuple. Shape of the input features.
        num_nodes: Int. Number of nodes in the hidden layers.

    Returns:
        model: ListNet neural network model.
    """

    model = Sequential([
        Dense(num_nodes, activation="leaky_relu", input_shape=input_shape, kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1)),
        Dense(num_nodes, activation="leaky_relu", kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1)),
        Dense(num_nodes, activation="leaky_relu", kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1)),
        Dense(1, activation="softmax", kernel_initializer=RandomUniform(minval=-0.1, maxval=0.1)) # output layer
    ])

    return model

In [ ]:
# Ranking influencers
inf_rank = {}
model = create_listnet_model(inf_brand_similarity.shape[-1], 10)
for _, row in inf_brand_similarity.iterrows():
    inf_rank[row['influencer']] = model.fit(row)
inf_rank